In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pdb

df = pd.read_csv('airline-passengers.csv')
timeseries = df[["Passengers"]].values.astype('float32')

# train-test split for time series
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 4
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 2000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        pdb.set_trace()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(timeseries) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(timeseries) * np.nan
    test_plot[train_size+lookback:len(timeseries)] = model(X_test)[:, -1, :]
# plot
plt.plot(timeseries)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()

> /var/folders/26/pd267j_136sfv9lw7lc9rcm40000gp/T/ipykernel_43672/2574444148.py(57)<module>()
     55     for X_batch, y_batch in loader:
     56         pdb.set_trace()
---> 57         y_pred = model(X_batch)
     58         loss = loss_fn(y_pred, y_batch)
     59         optimizer.zero_grad()



ipdb>  X_batch


tensor([[[178.],
         [163.],
         [172.],
         [178.]],

        [[199.],
         [199.],
         [184.],
         [162.]],

        [[199.],
         [184.],
         [162.],
         [146.]],

        [[148.],
         [148.],
         [136.],
         [119.]],

        [[135.],
         [125.],
         [149.],
         [170.]],

        [[313.],
         [318.],
         [374.],
         [413.]],

        [[218.],
         [230.],
         [242.],
         [209.]],

        [[133.],
         [114.],
         [140.],
         [145.]]])


In [7]:
y_train

tensor([[[118.],
         [132.],
         [129.],
         [121.]],

        [[132.],
         [129.],
         [121.],
         [135.]],

        [[129.],
         [121.],
         [135.],
         [148.]],

        [[121.],
         [135.],
         [148.],
         [148.]],

        [[135.],
         [148.],
         [148.],
         [136.]],

        [[148.],
         [148.],
         [136.],
         [119.]],

        [[148.],
         [136.],
         [119.],
         [104.]],

        [[136.],
         [119.],
         [104.],
         [118.]],

        [[119.],
         [104.],
         [118.],
         [115.]],

        [[104.],
         [118.],
         [115.],
         [126.]],

        [[118.],
         [115.],
         [126.],
         [141.]],

        [[115.],
         [126.],
         [141.],
         [135.]],

        [[126.],
         [141.],
         [135.],
         [125.]],

        [[141.],
         [135.],
         [125.],
         [149.]],

        [[135.],
   